In [ ]:
import os
if "COLAB_GPU" in os.environ:
  !pip install pyMuPDF
  !pip install tqdm
  !pip install accelerate
  !pip install bitsandbytes
  !pip install flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 60.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [ ]:
!pip uninstall -y torch torchvision torchaudio transformers sentenct-transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers sentence-transformers


Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Found existing installation: transformers 4.56.1
Uninstalling transformers-4.56.1:
  Successfully uninstalled transformers-4.56.1
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.6 MB/s eta 0:00:00
  

In [ ]:
import os
import requests
pdf_path = "human-nutrition-text.pdf"
if not os.path.exists(pdf_path):
  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
  filename = pdf_path
  response = requests.get(url)
  if response.status_code == 200:
    with open(filename, 'wb') as f:
      f.write(response.content)
  else:
    print(f"Failed to download the file. Status code: {response.status_code}")
else:
  print("file {pdf_path} exists.")

In [ ]:
import fitz
from tqdm.auto import tqdm
def text_formatter(text: str)-> str:
  cleaned_text = text.replace("\n","").strip()
  return cleaned_text
def open_and_read_pdf(pdf_path: str)-> list[dict]:
  doc = fitz.open(pdf_path)
  pages_and_texts = []
  for page_number,page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text)
    pages_and_texts.append({"page_number": page_number-41,
                            "page_char_count": len(text),
                            "page_word_count": len(text.split(". ")),
                            "page_sentence_count_raw": len(text.split(". ")),
                            "page_token_count": len(text)/4,
                            "text":text})
  return pages_and_texts
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [ ]:
import random
random.sample(pages_and_texts,k=3)

[{'page_number': 805,
  'page_char_count': 873,
  'page_word_count': 7,
  'page_sentence_count_raw': 7,
  'page_token_count': 218.25,
  'text': 'Learning Activities Technology Note: The second edition of the Human Nutrition Open Educational Resource (OER) textbook features interactive learning activities.\xa0 These activities are available in the web-based textbook and not available in the downloadable versions (EPUB, Digital PDF, Print_PDF, or Open Document). Learning activities may be used across various mobile devices, however, for the best user experience it is strongly recommended that users complete these activities using a desktop or laptop computer and in Google Chrome. \xa0An interactive or media element has been excluded from this version of the text. You can view it online here: http://pressbooks.oer.hawaii.edu/humannutrition2/?p=446 gestational diabetes: The influence of changing diagnostic criteria. World Journal of Diabetes, 6(2), 234–244. https://doi.org/10.4239/wjd.v6.i

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,1,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,308,1,1,77.00,Human Nutrition: 2020 Edition UNIVERSITY OF HA...
3,-38,210,1,1,52.50,Human Nutrition: 2020 Edition by University of...
4,-37,766,2,2,191.50,Contents Preface University of Hawai‘i at Māno...


In [ ]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1122.01,9.97,9.97,280.50
std,348.86,551.68,6.18,6.18,137.92
min,-41.00,0.00,1.00,1.00,0.00
25%,260.75,744.00,4.00,4.00,186.00
50%,562.50,1194.00,10.00,10.00,298.50
75%,864.25,1571.25,14.00,14.00,392.81
max,1166.00,2273.00,32.00,32.00,568.25


## Chunking

In [ ]:
def chunk_text(text:str,chunk_size: int =500)->list:
  chunks = []
  current_chunk = ""
  words = text.split()
  for word in words:
    if len(current_chunk) + len(word) + 1 <= chunk_size:
      current_chunk += (word+' ')
    else:
      chunks.append(current_chunk.strip())
      current_chunk = word + ' '
  if current_chunk:
    chunks.append(current_chunk.strip())
  return chunks
def chunk_pdf_pages(pages_and_text: list,chunk_size: int = 500)-> list[dict]:
  all_chunks = []
  for page in tqdm(pages_and_texts):
    page_number = page['page_number']
    page_text = page['text']
    chunks = chunk_text(page_text,chunk_size=chunk_size)
    for i, chunk in enumerate(chunks):
      all_chunks.append({
          "page_number": page_number,
          "chunk_index": i,
          "chunk_char_count": chunk,
          "chunk_word_count": len(chunk.split()),
          "chunk_token_count": len(chunk)/4,
          "chunk_text": chunk
      })
  return all_chunks
chunked_pages = chunk_pdf_pages(pages_and_texts,chunk_size=500)
print(f"Number of Chunks: {len(chunked_pages)}")
print(f"First chunk (page{chunked_pages[0]['page_number']}): {chunked_pages[0]['chunk_text'][:200]}...")

  0%|          | 0/1208 [00:00<?, ?it/s]

Number of Chunks: 3322
First chunk (page-41): Human Nutrition: 2020 Edition...


## Fixed_size chunking

In [ ]:
import random, textwrap
def _scattered_indices(n: int,k:int,jitter_frac:float = 0.08)->list[int]:
  if k<=0:
    return []
  if k == 1:
    return[random.randrange(n)]
  anchors = [int(round(i*(n-1)/(k-1))) for i in range(k)]
  out,seen = [], set()
  radius = max(1,int(n*jitter_frac))
  for a in anchors:
    lo, hi = max(0,a-radius), min(n-1,a+radius)
    j = random.randint(lo,hi)
    if j not in seen:
      out.append(j)
      seen.add(j)
  while len(out) < k:
    r = random.randrange(n)
    if r not in seen:
      out.append(r)
      seen.add(r)
  return out

def _draw_boxed_chunk(c: dict, wrap_at: int = 96) -> str:
    header = (
        f" Chunk p({c['page_number']}) · idx ({c['chunk_index']}) | "
        f"chars ({c['chunk_char_count']}) · words ({c['chunk_word_count']}) · ~tokens ({c['chunk_token_count']})"
    )
    # Wrap body text, avoid breaking long words awkwardly
    wrapped_lines = textwrap.wrap(
        c["chunk_text"], width=wrap_at, break_long_words=False, replace_whitespace=False
    )
    content_width = max([0, *map(len, wrapped_lines)])
    box_width = max(len(header), content_width + 2) # +2 for side padding

    top    = "+" + "-" * box_width + "+"
    hline  = "|" + header.ljust(box_width) + "|"
    sep    = "+" + "-" * box_width + "+"
    body   = "\n".join(["| " + line.ljust(box_width - 2) + " |" for line in wrapped_lines] or \
                     ["| " + "".ljust(box_width - 2) + " |"])
    bottom = "+" + "-" * box_width + "+"
    return "\n".join([top, hline, sep, body, bottom])


def show_random_chunks(pages_and_texts: list, chunk_size: int = 500, k: int = 5, seed: int | None = 42):
    if seed is not None:
        random.seed(seed)
    all_chunks = chunk_pdf_pages(pages_and_texts, chunk_size=chunk_size)
    if not all_chunks:
        print("No chunks to display.")
        return

    idxs = _scattered_indices(len(all_chunks), k)
    print(f"Showing {len(idxs)} scattered random chunks out of {len(all_chunks)} total:\n")
    for i, idx in enumerate(idxs, 1):
        print(f"#{i}")
        print(_draw_boxed_chunk(all_chunks[idx]))
        print()

# ---------- Run ----------
assert 'pages_and_texts' in globals(), "Run: pages_and_texts = open_and_read_pdf(pdf_path) first."
show_random_chunks(pages_and_texts, chunk_size=500, k=5, seed=42)

  0%|          | 0/1208 [00:00<?, ?it/s]

Showing 5 scattered random chunks out of 3322 total:

#1
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Chunk p(-9) · idx (0) | chars (Skylar Hara Skylar Hara is an undergraduate student student in the Tropical Agriculture and the Environment program at the University of Hawai‘i at Mānoa. She has a growing love for plants and hopes to go to graduate school to conduct research in the future. About the Contributors | xxxiii) · words (49) · ~tokens (72.5)|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
nltk.download('punkt', quiet=True)

# Load once globally
semantic_model = SentenceTransformer("all-MiniLM-L6-v2")

def semantic_chunk_text(text: str, similarity_threshold: float = 0.8, max_tokens: int = 500) -> list:
    """
    Splits text into semantic chunks based on sentence similarity and max token length.
    """
    sentences = nltk.sent_tokenize(text)
    if not sentences:
        return []

    embeddings = semantic_model.encode(sentences)

    chunks = []
    current_chunk = [sentences[0]]
    current_embedding = embeddings[0]

    for i in range(1, len(sentences)):
        sim = cosine_similarity([current_embedding], [embeddings[i]])[0][0]
        chunk_token_count = len(" ".join(current_chunk)) // 4

        if sim >= similarity_threshold and chunk_token_count < max_tokens:
            current_chunk.append(sentences[i])
            current_embedding = np.mean([current_embedding, embeddings[i]], axis=0)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentences[i]]
            current_embedding = embeddings[i]

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from tqdm.auto import tqdm

def semantic_chunk_pdf_pages(pages_and_texts: list,
                             similarity_threshold: float = 0.8,
                             max_tokens: int = 500) -> list[dict]:
    """
    Takes PDF pages with text and splits them into semantic chunks.

    Returns a list of dicts with page_number, chunk_index, and chunk_text.
    """
    all_chunks = []

    for page in tqdm(pages_and_texts, desc="Semantic chunking pages"):
        page_number = page["page_number"]
        page_text = page["text"]

        chunks = semantic_chunk_text(page_text,
                                     similarity_threshold=similarity_threshold,
                                     max_tokens=max_tokens)

        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk) // 4,  # rough token estimate
                "chunk_text": chunk
            })

    return all_chunks

In [ ]:
import nltk
nltk.download('punkt_tab')
semantic_chunked_pages = semantic_chunk_pdf_pages(pages_and_texts,
                                                  similarity_threshold=0.75,
                                                  max_tokens=500)

print(f"Total semantic chunks: {len(semantic_chunked_pages)}")
print(f"First semantic chunk (page {semantic_chunked_pages[0]['page_number']}):")
print(semantic_chunked_pages[0]['chunk_text'][:200] + "...")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Semantic chunking pages:   0%|          | 0/1208 [00:00<?, ?it/s]

Total semantic chunks: 12016
First semantic chunk (page -41):
Human Nutrition: 2020 Edition...


In [ ]:
# Pretty-print 5 random SEMANTIC chunks (uses `semantic_chunked_pages` from your code above)
import random
import textwrap

def _scattered_indices(n: int, k: int, jitter_frac: float = 0.08) -> list[int]:
    """Evenly spaced anchors + random jitter -> indices scattered across [0, n-1]."""
    if k <= 0:
        return []
    if k == 1:
        return [random.randrange(n)]

    anchors = [int(round(i * (n - 1) / (k - 1))) for i in range(k)]
    out, seen = [], set()
    radius = max(1, int(n * jitter_frac))
    for a in anchors:
        lo, hi = max(0, a - radius), min(n - 1, a + radius)
        j = random.randint(lo, hi)
        if j not in seen:
            out.append(j); seen.add(j)

    while len(out) < k:
        r = random.randrange(n)
        if r not in seen:
            out.append(r); seen.add(r)

    return out

In [ ]:
def _draw_boxed_chunk(c: dict, wrap_at: int = 96) -> str:
    approx_tokens = c.get('chunk_token_count', len(c.get('chunk_text', ''))/4)
    header = (
        f" Chunk p({c['page_number']}) · idx ({c['chunk_index']}) | "
        f"chars ({c['chunk_char_count']}) · words ({c['chunk_word_count']}) · ~tokens {round(approx_tokens, 2)}"
    )
    wrapped_lines = textwrap.wrap(
        c["chunk_text"], width=wrap_at, break_long_words=False, replace_whitespace=False
    )
    content_width = max([0, *map(len, wrapped_lines)])
    box_width = max(len(header), content_width + 2) # +2 for side padding

    top    = "+" + "-" * box_width + "+"
    hline  = "|" + header.ljust(box_width) + "|"
    sep    = "+" + "-" * box_width + "+"
    body   = "\n".join(["| " + line.ljust(box_width - 2) + " |" for line in wrapped_lines] or \
                     ["| " + "".ljust(box_width - 2) + " |"])
    bottom = "+" + "-" * box_width + "+"
    return "\n".join([top, hline, sep, body, bottom])


def show_random_semantic_chunks(semantic_chunked_pages: list[dict], k: int = 5, seed: int | None = 42):
    if seed is not None:
        random.seed(seed)

    n = len(semantic_chunked_pages)
    if n == 0:
        print("No semantic chunks to display.");
        return

    idxs = _scattered_indices(n, k)
    print(f"Showing {len(idxs)} scattered random SEMANTIC chunks out of {n} total:\n")
    for i, idx in enumerate(idxs, 1):
        print(f"#{i}")
        print(_draw_boxed_chunk(semantic_chunked_pages[idx]))
        print()

# -- Run (expects you've already created `semantic_chunked_pages`) ---
assert 'semantic_chunked_pages' in globals() and len(semantic_chunked_pages) > 0, \
    "Run your semantic chunking code first to define `semantic_chunked_pages`."
show_random_semantic_chunks(semantic_chunked_pages, k=5, seed=42)

Showing 5 scattered random SEMANTIC chunks out of 12016 total:

#1
+--------------------------------------------------------------------------------------------------+
| Chunk p(56) · idx (7) | chars (197) · words (31) · ~tokens 49                                    |
+--------------------------------------------------------------------------------------------------+
| Observing the connection between the beverage and longevity, Dr. Elie Metchnikoff began his      |
| research on beneficial bacteria and the longevity of life that led to his book, The Prolongation |
| of Life.                                                                                         |
+--------------------------------------------------------------------------------------------------+

#2
+--------------------------------------------------------------------------------------------------+
| Chunk p(232) · idx (6) | chars (244) · words (36) · ~tokens 61                                   |
+-------------------

Recursive Chunking

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
from tqdm.auto import tqdm
nltk.download("punkt")

def recursive_chunk_text(text: str, max_chunk_size: int = 1000, min_chunk_size: int = 100) -> list:
    """
    Recursively splits a block of text into chunks that fit within size constraints.
    Tries splitting by sections, then newlines, then sentences.
    """
    def split_chunk(chunk: str) -> list:
        # Base case
        if len(chunk) <= max_chunk_size:
            return [chunk]

        # Try splitting by double newlines
        sections = chunk.split("\n\n")
        if len(sections) > 1:
            result = []
            for section in sections:
                if section.strip():
                    result.extend(split_chunk(section.strip()))
            return result

        # Try splitting by single newline
        sections = chunk.split("\n")
        if len(sections) > 1:
            result = []
            for section in sections:
                if section.strip():
                    result.extend(split_chunk(section.strip()))
            return result

        # Fallback: split by sentences
        sentences = nltk.sent_tokenize(chunk)
        chunks, current_chunk, current_size = [], [], 0

        for sentence in sentences:
            if current_size + len(sentence) > max_chunk_size:
                if current_chunk:
                    chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_size = len(sentence)
            else:
                current_chunk.append(sentence)
                current_size += len(sentence)

        if current_chunk:
            chunks.append(" ".join(current_chunk))

        return chunks

    return split_chunk(text)





def recursive_chunk_pdf_pages(pages_and_texts: list,
                              max_chunk_size: int = 1000,
                              min_chunk_size: int = 100) -> list[dict]:
    """
    Takes PDF pages with text and splits them into recursive chunks.

    Returns a list of dicts with page_number, chunk_index, and chunk_text.
    """
    all_chunks = []

    for page in tqdm(pages_and_texts, desc="Recursive chunking pages"):
        page_number = page["page_number"]
        page_text = page["text"]

        chunks = recursive_chunk_text(page_text,
                                      max_chunk_size=max_chunk_size,
                                      min_chunk_size=min_chunk_size)

        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk) / 4, # rough token estimate
                "chunk_text": chunk
            })

    return all_chunks

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import random
import textwrap

# --- Code to process and chunk the PDF text ---

recursive_chunked_pages = recursive_chunk_pdf_pages(pages_and_texts,
                                                    max_chunk_size=800,
                                                    min_chunk_size=100)

print(f"Total recursive chunks: {len(recursive_chunked_pages)}")
print(f"First recursive chunk (page {recursive_chunked_pages[0]['page_number']}):")
print(recursive_chunked_pages[0]['chunk_text'][:200] + "...")


# --- Functions to display random chunks ---

# Pretty-print 5 random RECURSIVE chunks (uses `recursive_chunked_pages` from your code above)
def _scattered_indices(n: int, k: int, jitter_frac: float = 0.08) -> list[int]:
    """Evenly spaced anchors + random jitter -> indices scattered across [0, n-1]."""
    if k <= 0:
        return []
    if k == 1:
        return [random.randrange(n)]

    anchors = [int(round(i * (n - 1) / (k - 1))) for i in range(k)]
    out, seen = [], set()
    radius = max(1, int(n * jitter_frac))
    for a in anchors:
        lo, hi = max(0, a - radius), min(n - 1, a + radius)
        j = random.randint(lo, hi)
        if j not in seen:
            out.append(j); seen.add(j)

    while len(out) < k:
        r = random.randrange(n)
        if r not in seen:
            out.append(r); seen.add(r)

    return out


def show_random_recursive_chunks(recursive_chunked_pages: list[dict], k: int = 5, seed: int | None = 42):
    if seed is not None:
        random.seed(seed)

    n = len(recursive_chunked_pages)
    assert n > 0, "No recursive chunks to display. Did you run the recursive chunking cell?"

    idxs = _scattered_indices(n, k)
    print(f"Showing {len(idxs)} scattered random RECURSIVE chunks out of {n} total:\n")
    for i, idx in enumerate(idxs, 1):
        print(f"#{i}")
        print(_draw_boxed_chunk(recursive_chunked_pages[idx]))
        print()

def _draw_boxed_chunk(c: dict, wrap_at: int = 96) -> str:
    approx_tokens = c.get('chunk_token_count', len(c.get('chunk_text', ''))/4)
    header = (
        f" Chunk p({c['page_number']}) · idx ({c['chunk_index']}) | "
        f"chars ({c['chunk_char_count']}) · words ({c['chunk_word_count']}) · ~tokens {round(approx_tokens, 2)}"
    )
    wrapped_lines = textwrap.wrap(
        c["chunk_text"], width=wrap_at, break_long_words=False, replace_whitespace=False
    )
    content_width = max([0, *map(len, wrapped_lines)])
    box_width = max(len(header), content_width + 2) # +2 for side padding

    top    = "+" + "-" * box_width + "+"
    hline  = "|" + header.ljust(box_width) + "|"
    sep    = "+" + "-" * box_width + "+"
    body   = "\n".join(["| " + line.ljust(box_width - 2) + " |" for line in wrapped_lines] or \
                     ["| " + "".ljust(box_width - 2) + " |"])
    bottom = "+" + "-" * box_width + "+"
    return "\n".join([top, hline, sep, body, bottom])


# --- Run (expects you've already created `recursive_chunked_pages`) ---
assert 'recursive_chunked_pages' in globals() and len(recursive_chunked_pages) > 0, \
    "Run your recursive chunking code first to define `recursive_chunked_pages`."
show_random_recursive_chunks(recursive_chunked_pages, k=5, seed=42)

Recursive chunking pages:   0%|          | 0/1208 [00:00<?, ?it/s]

Total recursive chunks: 2401
First recursive chunk (page -41):
Human Nutrition: 2020 Edition...
Showing 5 scattered random RECURSIVE chunks out of 2401 total:

#1
+------------------------------------------------------------+
| Chunk p(54) · idx (0) | chars (0) · words (0) · ~tokens 0.0|
+------------------------------------------------------------+
|                                                            |
+------------------------------------------------------------+

#2
+--------------------------------------------------------------------------------------------------+
| Chunk p(212) · idx (1) | chars (518) · words (75) · ~tokens 129.5                                |
+--------------------------------------------------------------------------------------------------+
| Sources of Drinking Water The Beverage Panel recommends that women drink at least 32 ounces and  |
| men drink at least 48 ounces of water daily. In 1974, the US federal government enacted The Safe |
| Drinking Wa

Structured Chunking

In [ ]:
import re
import random
import textwrap

# 1) Helper to detect "chapter start" pages
def _is_chapter_header_page(text: str) -> bool:
    # Robust to punctuation/diacritics differences; matches the recurring header
    # e.g., "UNIVERSITY OF HAWAI'I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM"
    has_header = re.search(r"university\s+of\s+hawai['‘]?i", text, flags=re.IGNORECASE) is not None
    is_short_page = len(text) < 1500

    #return re.search(r"university\s+of\s+hawaii", text, flags=re.IGNORECASE) is not None
    return has_header and is_short_page

def _guess_title_from_page(text: str) -> str:
    """
    Best-effort chapter title guess = the text before the 'University of Hawaii' header line.
    Falls back to the first ~120 characters.
    """
    m = re.search(r"university\s+of\s+hawaii", text, flags=re.IGNORECASE)
    if m:
        title = text[:m.start()].strip()
        # keep it readable
        title = re.sub(r"\s+", " ", title).strip()
        if 10 <= len(title) <= 180:
            return title

    # fallback
    t = re.sub(r"\s+", " ", text).strip()
    return t[:120] if t else "Untitled Chapter"

# 2) Build chapter chunks
def chapter_chunk_pdf_pages(pages_and_texts: list[dict]) -> list[dict]:
    """
    Returns a list of chapter chunks:
    [
        {
            'chapter_index': int,
            'title': str,
            'page_start': int,  # adjusted page number (your -41 offset)
            'page_end': int,
            'chunk_char_count': int,
            'chunk_word_count': int,
            'chunk_token_count': float, # ~chars/4
            'chunk_text': str
        }, ...
    ]
    """
    if not pages_and_texts:
        return []

    # Find all page indices that look like the start of a chapter
    chapter_starts = []
    for i, p in enumerate(pages_and_texts):
        txt = p["text"]
        if _is_chapter_header_page(txt):
            chapter_starts.append(i)

    # If nothing detected, return empty (or treat entire doc as one chunk)
    if not chapter_starts:
        # Treat entire doc as one "chapter"
        all_text = " ".join(p["text"] for p in pages_and_texts).strip()
        return [{
            "chapter_index": 0,
            "title": _guess_title_from_page(pages_and_texts[0]["text"]),
            "page_start": pages_and_texts[0]["page_number"],
            "page_end": pages_and_texts[-1]["page_number"],
            "chunk_char_count": len(all_text),
            "chunk_word_count": len(all_text.split()),
            "chunk_token_count": round(len(all_text) / 4, 2),
            "chunk_text": all_text
        }]

    # Build chapter ranges (start -> next_start-1)
    chapter_chunks = []
    for ci, s in enumerate(chapter_starts):
        e = (chapter_starts[ci + 1] - 1) if (ci + 1 < len(chapter_starts)) else (len(pages_and_texts) - 1)
        if e < s:
            continue # guard (shouldn't happen)

        pages = pages_and_texts[s:e + 1]
        text_concat = " ".join(p["text"] for p in pages).strip()
        title = _guess_title_from_page(pages[0]["text"])

        chapter_chunks.append({
            "chapter_index": ci,
            "title": title,
            "page_start": pages[0]["page_number"],
            "page_end": pages[-1]["page_number"],
            "chunk_char_count": len(text_concat),
            "chunk_word_count": len(text_concat.split()),
            "chunk_token_count": round(len(text_concat) / 4, 2),
            "chunk_text": text_concat
        })

    return chapter_chunks

In [ ]:
structure_chunked_pages = chapter_chunk_pdf_pages(pages_and_texts)

print(f"Total chapter-based chunks: {len(structure_chunked_pages)}")
if structure_chunked_pages:
    first = structure_chunked_pages[0]
    print(f"First chapter (pages {first['page_start']}-{first['page_end']}): {first['title']}")
    print(first['chunk_text'][:200] + "...")
else:
    print("No chapters detected.")

Total chapter-based chunks: 129
First chapter (pages -39--39): Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SK
Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FI...


In [ ]:
# 3) (Optional) Pretty print a few chapter chunks to inspect
def _draw_boxed_chunk(c: dict, wrap_at: int = 96) -> str:
    header = (
        f" chapter ({c['chapter_index']}) | {c['title'][:120]}"
        f" p({c['page_start']}-{c['page_end']}) | ~tokens ({c['chunk_token_count']})"
    )
    wrapped = textwrap.wrap(c['chunk_text'], width=wrap_at, break_long_words=False, replace_whitespace=False)
    content_width = max(len(header), *(len(x) for x in wrapped)) if wrapped else len(header)

    top    = "+" + "-"*content_width + "+"
    hline  = "|" + header.ljust(content_width) + "|"
    sep    = "+" + "-"*content_width + "+"
    body   = "\n".join("| " + line.ljust(content_width-2) + " |" for line in wrapped[:12]) or \
             ("| " + "".ljust(content_width-2) + " |")
    bottom = "+" + "-"*content_width + "+"
    return "\n".join([top, hline, sep, body, bottom])


def show_random_chapter_chunks(chapter_chunks: list[dict], k: int = 5, seed: int | None = 42):
    if not chapter_chunks:
        print("No chapter chunks to display."); return
    if seed is not None:
        random.seed(seed)

    k = min(k, len(chapter_chunks))
    idxs = random.sample(range(len(chapter_chunks)), k)

    print(f"Showing {k} random chapters out of {len(chapter_chunks)} total:\n")
    for i, idx in enumerate(idxs, 1):
        print(f"#{i}")
        print(_draw_boxed_chunk(chapter_chunks[idx]))
        print()


# --- Run the chapter chunking and display functions ---
chapter_chunks = chapter_chunk_pdf_pages(pages_and_texts)
print(f"Total chapters detected: {len(chapter_chunks)}")
if chapter_chunks:
    print(f"First chapter: {chapter_chunks[0]['title']} (p{chapter_chunks[0]['page_start']}-{chapter_chunks[0]['page_end']})")

    # Inspect a few
    show_random_chapter_chunks(chapter_chunks, k=5, seed=21)

Total chapters detected: 129
First chapter: Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SK (p-39--39)
Showing 5 random chapters out of 129 total:

#1
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| chapter (42) | Image by Henrique Felix on unsplash.com / CC0 Introduction UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITI p(55-81) | ~tokens (7169.5)|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Image by Henrique Felix on unsplash.com / CC0 Introduction UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD                                                                     |
| SCIENCE AND HUMAN NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM I ola no ke kino i ka māʻona o            

In [ ]:
import os
import google.generativeai as genai
from typing import List, Dict
from tqdm.auto import tqdm

import os
try:
    # Used in Google Colab
    from google.colab import userdata
    # Get the secret value by its NAME
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except ImportError:
    # Fallback for local environments
    # Get the environment variable by its NAME
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set it as a secret or environment variable.")

# Now, configure the library with the key you just loaded
genai.configure(api_key=GOOGLE_API_KEY)

if not GOOGLE_API_KEY:
    print("⚠️ Please set your GOOGLE_API_KEY environment variable or Colab secret.")
else:
    genai.configure(api_key=GOOGLE_API_KEY)


def gemini_based_chunk(text: str,
                       chunk_size: int = 1000,
                       model_name: str = "gemini-1.5-flash") -> List[str]:
    """
    Uses the Gemini model to find semantically coherent chunk boundaries.
    """
    # Initialize the Gemini model
    model = genai.GenerativeModel(model_name)

    def get_chunk_boundary(text_segment: str) -> int:
        """
        Ask the Gemini model where to split within this text segment.
        Returns an integer index within the text_segment.
        """
        prompt = f"""
        Analyze the following text and identify the best point to split it
        into two semantically coherent parts.
        The split should occur near {chunk_size} characters.

        Text:
        \"\"\"{text_segment}\"\"\"

        Return only the integer index (character position) where the split should occur.
        Do not return any explanation, code, or formatting. Just the number.
        """
        try:
            response = model.generate_content(prompt)
            # The response.text should be a string containing just a number
            split_point = int(response.text.strip())
            return split_point
        except (ValueError, AttributeError, Exception) as e:
            # If the model fails or returns non-numeric text, fall back to a hard split
            print(f"⚠️ Model failed to return a valid split point ({e}). Using hard split.")
            return chunk_size

    chunks = []
    remaining_text = text

    while len(remaining_text) > chunk_size:
        # Look ahead in the text to give the model context for the split
        text_window = remaining_text[:chunk_size * 2]

        split_point = get_chunk_boundary(text_window)

        # Safety check: ensure the split point is reasonable
        if not (100 < split_point < len(text_window) - 100):
            print(f"⚠️ Unreasonable split point ({split_point}). Using hard split.")
            split_point = chunk_size

        chunks.append(remaining_text[:split_point].strip())
        remaining_text = remaining_text[split_point:].strip()

    # Add the final remaining piece of text
    if remaining_text:
        chunks.append(remaining_text)

    return chunks


def gemini_based_chunk_pdf_pages(pages_and_texts: List[Dict],
                                 chunk_size: int = 1000,
                                 model: str = "gemini-1.5-flash") -> List[Dict]:
    """
    Applies Gemini-based chunking to each PDF page.
    Returns a list of dicts with page_number, chunk_index, and chunk_text.
    """
    all_chunks = []

    for page in tqdm(pages_and_texts, desc="LLM-based chunking pages"):
        page_number = page["page_number"]
        page_text = page["text"]

        chunks = gemini_based_chunk(page_text, chunk_size=chunk_size, model_name=model)
        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk) / 4,  # rough estimate
                "chunk_text": chunk
            })

    return all_chunks

In [ ]:
# Run the Gemini-powered chunking on your document
gemini_chunked_pages = gemini_based_chunk_pdf_pages(pages_and_texts,
                                                    chunk_size=1000,
                                                    model="gemini-1.5-pro")

# Print the total number of chunks created
print(f"Total Gemini-based chunks: {len(gemini_chunked_pages)}")

# You can now use your existing functions to display random chunks
# For example, if you have a `show_random_chunks` function:
show_random_chunks(gemini_chunked_pages)

LLM-based chunking pages:   0%|          | 0/1208 [00:00<?, ?it/s]

⚠️ Model failed to return a valid split point (400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: API Key not found. Please pass a valid API key.). Using hard split.
⚠️ Unreasonable split point (1000). Using hard split.


⚠️ Model failed to return a valid split point (400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: API Key not found. Please pass a valid API key.). Using hard split.
⚠️ Unreasonable split point (1000). Using hard split.


⚠️ Model failed to return a valid split point (400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: API Key not found. Please pass a valid API key.). Using hard split.
⚠️ Unreasonable split point (1000). Using hard split.


⚠️ Model failed to return a valid split point (400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: API Key not found. Please pass a valid API key.). Using hard split.


...

⚠️ Model failed to return a valid split point (400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: API Key not found. Please pass a valid API key.). Using hard split.
⚠️ Unreasonable split point (1000). Using hard split.


⚠️ Model failed to return a valid split point (400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: API Key not found. Please pass a valid API key.). Using hard split.


⚠️ Model failed to return a valid split point (400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: API Key not found. Please pass a valid API key.). Using hard split.


⚠️ Model failed to return a valid split point (400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: API Key not found. Please pass a valid API key.). Using hard split.


KeyboardInterrupt: 

In [ ]:
!pip install google-generativeai